In [9]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit import BasicAer
from scipy.optimize import minimize

In [62]:
def hourglass_solver(profits, weights, probs, max_weight, p_total):
    theta_s = [2 * np.arccos(np.sqrt(1 - p)) for p in probs]
    
    def expectation_value(params):
        p = len(params) // 2
        beta_s = params[:p]
        gamma_s = params[p:]
        n_qubits = len(profits)
        p_total = len(beta_s)
        qubits = QuantumRegister(n_qubits)
        bits = ClassicalRegister(n_qubits)
        circuit = QuantumCircuit(qubits, bits)
        
        # Qubits initialisieren 
        for i in range(n_qubits):
            circuit.p(theta_s[i], qubits[i])

        # QAOA anwenden    
        for p in range(p_total):
            for i in range(n_qubits):
                circuit.rz(gamma_s[p] * weights[i] * 2, qubits[i])
            for i in range(n_qubits):
                phi = 2 * np.arcsin(np.sqrt(probs[i]))
                circuit.rz(np.pi, qubits[i])
                circuit.ry(np.pi - phi, qubits[i])
                circuit.rz(2 * beta_s[p], qubits[i])
                circuit.ry(phi, qubits[i])

        # Finalen Zustand messen
        for i in range(n_qubits):
            circuit.measure(qubits[i], bits[i])
        backend = BasicAer.get_backend("qasm_simulator")
        job = backend.run(transpile(circuit, backend), shots=n_qubits)
        results = job.result().get_counts(circuit)
        outcomes = list(process_outcomes(results, profits, weights, max_weight))
        res = max(outcomes, key=lambda x: x[1])
        return -1 * res[1]

    params = [1.0 for _ in range(2 * p_total)]

    res = minimize(expectation_value, params, method="COBYLA")
    
    return res

In [63]:
def process_outcomes(result_dict, profits, weights, max_weight):
    def bitstring_to_bitint(bitstring):
            return np.array([int(c) for c in bitstring])
    outcome_bits = [bitstring_to_bitint(o) for o in list(result_dict.keys())]
    outcomes = [(o * np.array(profits)).sum() * int((o * np.array(weights)).sum() <= max_weight) for o in outcome_bits]
    return zip(list(result_dict.keys()), outcomes)

In [66]:
profits = [1.0 for _ in range(24)]
weights = [1.0 for _ in range(24)]
max_weight = 24
p_total = 2
probs = [0.5 for _ in range(24)]
hourglass_solver(profits, weights, probs, max_weight, p_total)

     fun: -24.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 32
  status: 1
 success: True
       x: array([1.98723778, 0.26253391, 1.00182317, 1.25      ])